I duplicate comments that occur below:
you can find list of used names in repo by datasets/partial_datasets/confusing_names.csv
you can find generated results in repo by datasets/partial_datasets/confusing_generated.zip

It's code from post below, i played with loading in 4bits and 8bits and stayed with 4,
wrote custom prompt after like 5 tries and manual checks,
didn't come up with any stategy that will be suitable for easy
annotation except of wrapping specific elevation in ||,
if elevation name doesn't match with the name i provided i don't label it as a mountain,
i label it as elevation, i'm not sure if it's right choice but after extraction of data from
geonames i got an idea that 'mountain' and moutai .

In [ ]:
!pip install transformers pydantic bitsandbytes-cuda110 bitsandbytes

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import pandas as pd
from tqdm import tqdm
from typing import List
import spacy
import re
import json
import gc

nlp = spacy.load("en_core_web_sm")

model_dir = "/kaggle/input/llama-3.1/transformers/8b-instruct/2/"

# after manually comparing results of 4b vs 8b
# I chose to stay with 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(
    model_dir, 
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Adjust model configuration
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load CSV files
# you can find it in repo by datasets/partial_datasets/
# i think generating more such confusing names could improve performance in case of outliers like this
confusing_names_df = pd.read_csv("/kaggle/input/confusing-mount-names/confusing_names.csv")

confusing_name_samples = list(confusing_names_df["Mountain Name"])


In [ ]:
system_prompt = """
You are an advanced AI trained in natural language processing and synthetic data generation.
Your task is to read the following mountain name and generate 5 unique sentences using mountain name in context that sentence won't be about mountain.
After that generate 3 sentences about mountain using given mountain name that could be understood from context - don't mention words mountain or mount but you can mention mountain name.
After that 2 sentences about mountain using given mountain name. 
For each mountain name, provide a JSON output with the following structure:
- "sentences_not_about_mountain": [ array that contains 5 sentences which are strings]
- "sentences_about_mountain_implied_from_context": [ array that contains 3 sentences which are strings]
- "sentences_directly_about_mountain": [ array that contains 2 sentences which are strings]

Make sure to extract the exact string of the mountain name without any corrections or conversions.
For sentences about mountain implied from context and sentences directly about mountain highlight name of peak, mountain or range by setting "||" around it. Do not provide any explanations.
Only respond with the JSON structured data.

### Example 1:
Input:
'
Mount Adams
'

Output:
[
    {
        "sentences_not_about_mountain": [
            "Adams was thrilled to receive the promotion she had worked so hard for.",
            "The Adams family has been living in this town for generations, contributing to its growth.",
            "During the lecture, Professor Adams captivated the audience with her groundbreaking research.",
            "The bakery on Main Street is famous for its delicious Adams Apple Pie recipe.",
            "Adams signed the treaty, marking a significant turning point in the negotiations."
        ],
        "sentences_about_mountain_implied_from_context": [
            "||Adams|| stood tall in the distance, its silhouette painted against the twilight sky.",
            "The hikers marveled at the breathtaking view from ||Adams||, feeling a sense of accomplishment.",
            "At sunrise, the peak of ||Adams|| glowed with a warm golden hue, mesmerizing everyone at the base."
        ],
        "sentences_directly_about_mountain": [
            "||Mount Adams|| is a favorite destination for those seeking adventure and tranquility in nature.",
            "Legends say that ||Mount Adams|| has been a source of inspiration for poets and explorers alike."
        ]
    }
]
### Example 2:
Input:
'
Mount Wilson
'

Output:
[
    {
      "sentences_not_about_mountain": [
        "Wilson greeted everyone warmly at the family reunion, his smile lighting up the room.",
        "The town's new mayor, Wilson, promised to bring change and prosperity to the community.",
        "As the storm intensified, Wilson tightened his grip on the steering wheel, determined to get everyone home safely.",
        "Wilson was the lead detective on the case, tirelessly piecing together clues to solve the mystery.",
        "Her dog, Wilson, wagged his tail excitedly as she prepared his dinner."
      ],
      "sentences_about_mountain_implied_from_context": [
        "The sunrise over ||Wilson|| painted the horizon in hues of orange and gold, leaving the hikers speechless.",
        "Legends say that ||Wilson|| holds secrets from ancient times, guarded by the winds and stars.",
        "The view from the top of ||Wilson|| stretched endlessly, revealing a patchwork of forests and shimmering lakes."
      ],
      "sentences_directly_about_mountain": [
        "||Mount Wilson|| is known for its challenging trails that attract adventurers from around the world.",
        "The icy winds sweeping through ||Mount Wilson|| make it a formidable yet mesmerizing destination."
      ]
    }
]

Please continue with task and stop only after generating valid output for given mountain by user by outputting '### Output ends here.' don't forget this strict rule, strictly output valid JSON structure, never forget [] brackets. 
"""

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
import os

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# extracting needed json structure based on output structure defined by prompt
def extract_json_from_response(response):
    try:
        response = response[response.find("User:"):]
        response = response[:response.find("### Output ends here.")]
        actual_jsonlike = response[response.find("["):]
        match = re.search(r'(\[.*)', actual_jsonlike, re.DOTALL)
        if match:
            json_content = match.group(0).strip()
            if not json_content.endswith(']'):
                json_content += ']'
            return json.loads(json_content)
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")
        return []


# I left this function but basically it's not doing it's job
# no matter what combinations i tried
# i couldn't figure out how to write function that will work, 
def create_stopping_criteria(stop_words, tokenizer, device):
    class StoppingCriteriaSub(StoppingCriteria):
        def __init__(self, stops = [], device=device, encounters = 1):
            super().__init__()
            self.stops = stops = [stop.to(device) for stop in stops]

        def __call__(self, input_ids: LongTensor, scores: FloatTensor) -> bool:
            last_token = input_ids[0][-1]
            for stop in self.stops:
                if tokenizer.decode(stop) == tokenizer.decode(last_token):
                    return True
            return False

    stop_word_ids = [tokenizer(stop_word,
                               return_tensors="pt", 
                               add_special_tokens=False)["input_ids"].squeeze() 
                               for stop_word in stop_words]

    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_word_ids)])
    return stopping_criteria


stop_words_list = ["Output ends"]
stopping_criteria = None
if stop_words_list is not None:
    stopping_criteria = create_stopping_criteria(stop_words_list, tokenizer, device)

def write_batch_to_json(batch_entities, start_idx):
    filename = os.path.join("/kaggle/working/generated/", "batch_" + str(start_idx) + ".json")
    with open(filename, "w", encoding='utf-8') as f:
        json.dump(batch_entities, f)
        

def predict_entities_in_batches(test_dataset, model, tokenizer, system_prompt, batch_size=8):
    text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

    with torch.no_grad():
        for i in tqdm(range(0, len(test_dataset), batch_size), desc="Processing batches"):
            batch_entities = []
            if (i + batch_size) > len(test_dataset):
                batch_prompts = test_dataset[i:]
            else:
                batch_prompts = test_dataset[i:i + batch_size]

            chat_inputs = [
                f"{system_prompt}\nUser: {prompt}" for prompt in batch_prompts
            ]
            results = text_generation_pipeline(chat_inputs, max_new_tokens=400, do_sample=False, stopping_criteria=stopping_criteria)
            gc.collect()
            torch.cuda.empty_cache()
            for result in results:
                generated_text = result[0]['generated_text']
                entities = extract_json_from_response(generated_text)
                batch_entities.append(entities)
            
            write_batch_to_json(batch_entities, i) 

os.makedirs("/kaggle/working/generated/", exist_ok=True)
dataset = confusing_name_samples
processed_data = predict_entities_in_batches(dataset, model, tokenizer, system_prompt)

In [ ]:
import shutil
shutil.make_archive('/kaggle/working/generated', "zip", '/kaggle/working/generated/')